## define functions

- contains requered number -> ensure only specific images are pull
- get image -> pull the require and compress image
- scrapp_image_description -> iterate for each of the products
- grucci -> iterate scrap_image

### scrapper

In [1]:
import requests
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO
#import pandas as pd
from openpyxl import load_workbook
import re
#import psycopg2

import time

In [2]:
from DB_and_Azure import sql_db_functions as SQLf, azure_functions as Af

In [3]:
def create_soup(url):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    # Send an HTTP request to the URL
    response = requests.get(url, headers = header)
    soup = BeautifulSoup(response.content, 'html.parser')

    return soup


In [4]:
def contains_required_number(line):
    
    keeping = ['001_100','004_100','005_100']
    # 001: Prod
    # 004: back
    # 005: 3/4 image
    return any(num in line for num in keeping)

In [5]:
def get_image_links(prod_soup):
    
    item_list_1 = prod_soup.find_all(attrs={"data-image-size": "small-retina"})

    ##### get image
    images = []
    i = 0
    for item in item_list_1: 
            
        i+=1 
        if (i == len(item_list_1)) | (i == 1):
            continue
        elif not(contains_required_number(item['srcset'])) :
            continue
        else:
            images = images + [item['srcset']]
        
        i += 1

    return images

In [6]:
def get_description(prod_soup):
    
    description_list = prod_soup.find_all(attrs={"class": "product-detail"} )

    #Get description
    description = ''
    
    for a in description_list[0].ul.find_all('li'):
        description = description + ' ' + a.text
        

    return description_list[0].p.text.replace('\n', '').replace('\t', '') + ' ' + description

    

In [7]:
def links_to_images(links,prod_name,container_client):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    img_names = []

    i = 0
    for img_link in links:

        if i == 3:
            break
        
        else:

            try:
                response = requests.get('https:' + img_link, headers = header)
                img = Image.open(BytesIO(response.content))

                
                # Resize the image to 600
                img_resized = img.resize((500, 500))

                img_name = f"{prod_name}_{i}.jpg"

                # Load to Blob
                Af.images_to_blob(image = img_resized,image_name = img_name,container_client = container_client)
                
                
                i+=1

                img_names = img_names + [img_name]

                print(img_name)

            except Exception as e:
                # Rollback the transaction in case of error
                print("Error occurred:", e)

    return img_names
        

In [8]:
def get_price(prod_soup):

    text = prod_soup.select('span.price')[1].text


    # Remove any non-numeric characters except for ',' and '.'
    cleaned_text = re.sub(r'[^\d,\.]', '', text)
    
    # Replace comma with a period if there's no period already (to handle decimal part)
    if ',' in cleaned_text and '.' not in cleaned_text:
        cleaned_text = cleaned_text.replace(',', '.')
    elif ',' in cleaned_text and '.' in cleaned_text:
        # If both ',' and '.' are present, keep only the period as the decimal separator
        cleaned_text = cleaned_text.replace(',', '')
    
    # Convert the string to a float
    number = float(cleaned_text)
    
    return number

In [9]:
def gucci(category_url,n_products):

    soup = create_soup(category_url)
    item_list = soup.select('article a')

    container_client = Af.connect_blob()    

    br = 0
    for item in item_list: 
            
        if br == n_products:
            break
        
        try:

            #Scrapp
            prod_soup = create_soup('https://www.gucci.com' + item['href'])

            prod_images_links = get_image_links(prod_soup=prod_soup)
            prod_description = get_description(prod_soup=prod_soup)
            prod_price = get_price(prod_soup=prod_soup)

            prod_name = 'GUCCI_'+str(br)  

            print(f'start prod {br}')
            
            time.sleep(3)
            # Links to image, load to blob and return prod_images_names
            prod_images_names = links_to_images(links = prod_images_links,prod_name= prod_name,container_client=container_client)
            
            #print(prod_images_names)

            time.sleep(1)
            conn, cursor = SQLf.connect_sql()

            
            SQLf.insert_description_image_to_db(
                conn = conn,
                cursor= cursor, 
                brand='Gucci', 
                descript = prod_description, 
                price = prod_price,
                image_names = prod_images_names
                )
            
            conn.close()
            cursor.close()     
            time.sleep(1)

        except Exception as e:
            print(f"Error processing product {br}: {e}")     

        br += 1




## Execute Scrapper

In [10]:
# Define the URL you want to scrape
url = 'https://www.gucci.com/it/it/ca/women/ready-to-wear-for-women-c-women-readytowear'

gucci(url,3)

start prod 0
AZURE!! FUNCTION !!
GUCCI_0_0.jpg
AZURE!! FUNCTION !!
GUCCI_0_1.jpg
AZURE!! FUNCTION !!
GUCCI_0_2.jpg
['GUCCI_0_0.jpg', 'GUCCI_0_1.jpg', 'GUCCI_0_2.jpg']
['//media.gucci.com/style/DarkGray_Center_0_0_980x980/1710264633/786401_XKD2R_9288_001_100_0000_Light-Abito-a-pieghe-con-finiture-Web.jpg', '//media.gucci.com/style/DarkGray_Center_0_0_980x980/1710264635/786401_XKD2R_9288_004_100_0000_Light-Abito-a-pieghe-con-finiture-Web.jpg', '//media.gucci.com/style/DarkGray_Center_0_0_980x980/1710264635/786401_XKD2R_9288_005_100_0000_Light-Abito-a-pieghe-con-finiture-Web.jpg', '//media.gucci.com/style/White_Center_0_0_730x490/1710264633/786401_XKD2R_9288_001_100_0000_Light-Abito-a-pieghe-con-finiture-Web.jpg']
inserting
Brand_Prod_id generated: 4
Data inserted successfully
finished prod 0
start prod 1
AZURE!! FUNCTION !!
GUCCI_1_0.jpg
AZURE!! FUNCTION !!
GUCCI_1_1.jpg
AZURE!! FUNCTION !!
GUCCI_1_2.jpg
['GUCCI_1_0.jpg', 'GUCCI_1_1.jpg', 'GUCCI_1_2.jpg']
['//media.gucci.com/style/DarkGr